In [1]:
# from pyspark.context import SparkContext
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import StringIndexer, OneHotEncoder, StandardScaler, VectorAssembler 
from pyspark.ml import Pipeline
from pyspark.sql.functions import rand
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import DoubleType

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras import optimizers, regularizers
from keras.optimizers import Adam

from elephas.ml_model import ElephasEstimator

WARNING


In [2]:
conf = SparkConf().setAppName('Spark DL Tabular Pipeline').setMaster('local[*]')

# Increase memory spark has available
SparkContext.setSystemProperty('spark.executor.memory', '6g')

sc = SparkContext(conf=conf)

sql_context = SQLContext(sc)

#Verify settings set for available memory
sc._conf.getAll()

[('spark.executor.memory', '6g'),
 ('spark.app.id', 'local-1621799026203'),
 ('spark.driver.port', '38867'),
 ('spark.driver.host', '192.168.1.26'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.app.name', 'Spark DL Tabular Pipeline'),
 ('spark.app.startTime', '1621799025445')]

In [3]:
# Read the weather and the accident df

df_weather_temp = sql_context.read\
    .options(header=True, inferSchema=True)\
    .csv('datasets/hourly_weather.csv')

df_weather = df_weather_temp.withColumn('Hour', df_weather_temp.date[12:13][1:2].cast('double'))

df_accidents_temp = sql_context.read\
    .options(header=True, inferSchema=True)\
    .csv('datasets/accidents.csv')
df_accidents = df_accidents_temp.drop('Hour')

In [4]:
df_accidents.limit(5).toPandas()

,_c0,date,AccidentType,AccidentSeverityCategory,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,AccidentLocation_CHLV95_E,AccidentLocation_CHLV95_N,Month,WeekDay
0,0,2011-01-01 00:30:00,at0,as4,0,0,0,rt433,2684605,1245194,1,Saturday
1,1,2011-01-01 01:30:00,at0,as3,0,1,0,rt433,2682382,1246980,1,Saturday
2,2,2011-01-01 02:30:00,at0,as4,0,0,0,rt439,2682791,1247749,1,Saturday
3,3,2011-01-01 02:30:00,at5,as3,0,0,0,rt433,2681199,1247102,1,Saturday
4,4,2011-01-01 03:30:00,at0,as4,0,0,0,rt433,2682479,1250690,1,Saturday


In [5]:
df_weather.limit(5).toPandas()

,_c0,date,air_temperature,water_temperature,wind_gust_max_10min,wind_speed_avg_10min,wind_force_avg_10min,wind_direction,windchill,barometric_pressure_qfe,precipitation,dew_point,global_radiation,humidity,water_level,Hour
0,0,2011-01-01 00:30:00,2.233333,5.20,2.4,1.216667,1.216667,1785,2.20,974.55,0.0,1.616667,0.5,95.833333,67.635,0.0
1,1,2011-01-01 01:30:00,2.380000,5.20,2.8,0.860000,0.860000,1076,2.16,973.98,0.0,1.520000,0.6,93.600000,81.162,1.0
2,2,2011-01-01 02:30:00,2.580000,5.14,1.2,0.340000,0.340000,1159,2.58,973.64,0.0,1.380000,0.0,92.000000,81.162,2.0
3,3,2011-01-01 03:30:00,2.500000,5.16,1.9,0.520000,0.520000,1122,2.54,973.42,0.0,1.500000,0.0,92.800000,81.162,3.0
4,4,2011-01-01 04:30:00,2.500000,5.10,4.0,1.580000,1.580000,1108,1.62,973.16,0.0,0.720000,0.2,88.200000,81.162,4.0


In [6]:
df = df_weather.join(df_accidents, how='left')
# df.limit(5).toPandas()
# df.printSchema()

In [7]:
df = df.drop(*['AccidentLocation_CHLV95_N', 'AccidentLocation_CHLV95_E', 'RoadType',\
               'AccidentInvolvingMotorcycle', 'AccidentInvolvingBicycle', 'AccidentInvolvingPedestrian',\
              'AccidentType', 'date', '_c0'])
df.printSchema()

root
 |-- air_temperature: double (nullable = true)
 |-- water_temperature: double (nullable = true)
 |-- wind_gust_max_10min: double (nullable = true)
 |-- wind_speed_avg_10min: double (nullable = true)
 |-- wind_force_avg_10min: double (nullable = true)
 |-- wind_direction: integer (nullable = true)
 |-- windchill: double (nullable = true)
 |-- barometric_pressure_qfe: double (nullable = true)
 |-- precipitation: double (nullable = true)
 |-- dew_point: double (nullable = true)
 |-- global_radiation: double (nullable = true)
 |-- humidity: double (nullable = true)
 |-- water_level: double (nullable = true)
 |-- Hour: double (nullable = true)
 |-- AccidentSeverityCategory: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- WeekDay: string (nullable = true)



In [8]:
stages = []

X = df.columns
del X[-4:]
print(X)
# index accident type, accident severity and week day

severity_indexer = StringIndexer(inputCol='AccidentSeverityCategory', outputCol='severityIndex')
# severity_indexer = StringIndexer(inputCol='AccidentSeverityCategory', outputCol='severityIndex')
day_indexer = StringIndexer(inputCol='WeekDay', outputCol='weekDayIndex')

stages += [severity_indexer]
stages += [day_indexer]

# One-hot encoder for accident type, severity and week day

# ohe_accident_type = OneHotEncoder(inputCol='typeIndex', outputCol='type_vec')
# ohe_accident_severity = OneHotEncoder(intype_indexerputCol='severityIndex', outputCol='severity_vec')
ohe_weekday = OneHotEncoder(inputCol='weekDayIndex', outputCol='weekday_vec')
ohe_hour = OneHotEncoder(inputCol='Hour', outputCol='hour_vec')
5000
X += ['weekday_vec', 'hour_vec']
y = 'severityIndex'
print(X)
# stages += ohe_accident_type


assembler_final = VectorAssembler(inputCols=X, outputCol='features')

stages += [ohe_weekday, ohe_hour, assembler_final]

# Create pipeline and pass all stages
pipeline = Pipeline(stages=stages)

['air_temperature', 'water_temperature', 'wind_gust_max_10min', 'wind_speed_avg_10min', 'wind_force_avg_10min', 'wind_direction', 'windchill', 'barometric_pressure_qfe', 'precipitation', 'dew_point', 'global_radiation', 'humidity', 'water_level']
['air_temperature', 'water_temperature', 'wind_gust_max_10min', 'wind_speed_avg_10min', 'wind_force_avg_10min', 'wind_direction', 'windchill', 'barometric_pressure_qfe', 'precipitation', 'dew_point', 'global_radiation', 'humidity', 'water_level', 'weekday_vec', 'hour_vec']


In [9]:
stages

[StringIndexer_f66241805e7e,
 StringIndexer_09f3010dc298,
 OneHotEncoder_8f4e60a94084,
 OneHotEncoder_38b0b2494223,
 VectorAssembler_de09c458dd15]

In [10]:
# fit pipeline to data
pipeline_model = pipeline.fit(df)

# Transform data using fitted pipeline
df_transform = pipeline_model.transform(df)

In [11]:
# Inspect transformed data
df_transform.limit(5).toPandas()

,air_temperature,water_temperature,wind_gust_max_10min,wind_speed_avg_10min,wind_force_avg_10min,wind_direction,windchill,barometric_pressure_qfe,precipitation,dew_point,...,water_level,Hour,AccidentSeverityCategory,Month,WeekDay,severityIndex,weekDayIndex,weekday_vec,hour_vec,features
0,2.233333,5.2,2.4,1.216667,1.216667,1785,2.2,974.55,0.0,1.616667,...,67.635,0.0,as4,1,Saturday,0.0,5.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(2.233333333333334, 5.2, 2.4, 1.21666666666666..."
1,2.233333,5.2,2.4,1.216667,1.216667,1785,2.2,974.55,0.0,1.616667,...,67.635,0.0,as3,1,Saturday,1.0,5.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(2.233333333333334, 5.2, 2.4, 1.21666666666666..."
2,2.233333,5.2,2.4,1.216667,1.216667,1785,2.2,974.55,0.0,1.616667,...,67.635,0.0,as4,1,Saturday,0.0,5.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(2.233333333333334, 5.2, 2.4, 1.21666666666666..."
3,2.233333,5.2,2.4,1.216667,1.216667,1785,2.2,974.55,0.0,1.616667,...,67.635,0.0,as3,1,Saturday,1.0,5.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(2.233333333333334, 5.2, 2.4, 1.21666666666666..."
4,2.233333,5.2,2.4,1.216667,1.216667,1785,2.2,974.55,0.0,1.616667,...,67.635,0.0,as4,1,Saturday,0.0,5.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(2.233333333333334, 5.2, 2.4, 1.21666666666666..."


In [12]:
# select the features and the label for the final df
df_transform_fin = df_transform.select('features', 'severityIndex')
df_transform_fin.limit(5).toPandas()

,features,severityIndex
0,"(2.233333333333334, 5.2, 2.4, 1.21666666666666...",0.0
1,"(2.233333333333334, 5.2, 2.4, 1.21666666666666...",1.0
2,"(2.233333333333334, 5.2, 2.4, 1.21666666666666...",0.0
3,"(2.233333333333334, 5.2, 2.4, 1.21666666666666...",1.0
4,"(2.233333333333334, 5.2, 2.4, 1.21666666666666...",0.0


In [19]:
# Shuffle
df_transform_fin = df_transform_fin.orderBy(rand())

In [31]:
# Train / Test Split
train_data, test_data = df_transform_fin.randomSplit([.8, .2], seed=42)
train_data.persist()


DataFrame[features: vector, severityIndex: double]

In [32]:
# Number of classes
# too computationally expensive:
# nb_classes = train_data.select('severityIndex').distinct().count()
nb_classes = 4

In [ ]:
train_data.select('features').limit(5).toPandas()

In [ ]:
df.select('AccidentSeverityCategory').limit(5).toPandas()

In [33]:
# Input dimention
input_dim = len(train_data.select('features').first()[0])

KeyboardInterrupt: 

In [ ]:
model = Sequential()
model.add(Dense(256, input_shape=(input_dim,), activity_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(rate=0.3))

model.add(Dense(256, input_shape=(input_dim,), activity_regularizer=regularizers.l2(0.01)))
model.add(Activation('relu'))
model.add(Dropout(rate=0.3))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model.summary()

In [ ]:
# Set and Serial optimizer
optimizer_conf = optimizers.Adam(lr=0.01)
opt_conf = optimizers.serialize(optimizer_conf)

# Initialize SparkML Estimator and get Settings
estimator = ElephasEstimator()
estimator.setFeaturesCol('features')
estimator.setLabelCol('severityIndex')
estimator.set_keras_model_config(model.to_yaml())
estimator.set_categorical_labels(True)
estimator.set_nb_classes(nb_classes)
estimator.set_num_workers(1)
estimator.set_epochs(25)
estimator.set_batch_size(64)
estimator.set_verbosity(1)
estimator.set_validation_split(0.1)
estimator.set_optimizer_config(opt_conf)
estimator.set_mode('synchronous')
estimator.set_loss('categorical_crossentropy')
estimator.set_metrics(['acc'])

In [ ]:
# Create deep learning pipeline
dl_pipeline = Pipeline(stages=[estimator])

In [ ]:
# Helper function for fitting, transforming and predicting train and test
def dl_pip_fit_score_res(dl_pipeline=dl_pipeline, train_data=train_data,
                        test_data=test_data, label='labelIndex'):
    fit_dl_pipeline = dl_pipeline.fit(train_data)
    pred_train = fit_dl_pipeline.transform(train_data)
    pred_test = fit_dl_pipeline.transform(test_data)
    
    pnl_train = pred_train.select(label, 'prediction')
    pnl_test = pred_test.select(label, 'prediction')
    
    pred_and_label_train = pnl_train.rdd.map(lamda row: (row[label], row['prediction']))
    pred_and_label_test = pnl_test.rdd.map(lamda row: (row[label], row['prediction']))
    
    metrics_train = MulticlassMetrics(pred_and_label_train)
    metrics_test = MulticlassMetrics(pred_and_label_test)
    
    print('Train data accuracy: {}'.format(round(metrics_train.precision(), 4)))
    print('Training Data Confusion Matrix')
    display(pnl_train.crosstab('labelIndex', 'prediction\n').toPandas())
    
    print('Test data accuracy: {}'.format(round(metrics_test.precision(), 4)))
    print('Test Data Confusion Matrix')
    display(pnl_test.crosstab('labelIndex', 'prediction').toPandas())
    

In [ ]:
dl_pip_fit_score_res(dl_pipeline=dl_pipeline, train_data=train_data,
                        test_data=test_data, label='labelIndex')